In [ ]:
!pip install kagglehub --quiet

import os
import pandas as pd
import kagglehub


In [ ]:
# This will download the dataset into KaggleHub cache automatically
dataset_path = kagglehub.dataset_download("orvile/coughvid-v3")

print("Dataset cached at:", dataset_path)


100%|██████████| 2.15G/2.15G [00:22<00:00, 103MB/s]

Extracting files...


Dataset cached at: /root/.cache/kagglehub/datasets/orvile/coughvid-v3/versions/2


In [ ]:
# Paths inside the KaggleHub cache
base_path = dataset_path  # path returned by KaggleHub
audio_path = os.path.join(base_path, "public_dataset_v3/coughvid_20211012")
meta_path = os.path.join(base_path, "tabular_form")  # metadata CSVs live here

# List available files
print("Files in metadata folder:", os.listdir(meta_path))


Files in metadata folder: ['tabular_form']


In [ ]:
import os
import pandas as pd

# Paths
base_path = "/root/.cache/kagglehub/datasets/orvile/coughvid-v3/versions/2"
audio_path = os.path.join(base_path, "public_dataset_v3/coughvid_20211012")
metadata_file = os.path.join(base_path, "tabular_form/tabular_form/coughvid_v3.csv")

# Load metadata
df = pd.read_csv(metadata_file)

# Keep only WAV files (ignore .webm if any)
df = df[df["audio_name"].str.endswith(".wav")].copy()

# Add full audio paths
df["filepath"] = df["audio_name"].apply(lambda x: os.path.join(audio_path, x))

# Quick check
print(df.head())
print("Total recordings:", len(df))
print("Status counts:")
print(df["status"].value_counts())


                            datetime  cough_detected  latitude  longitude  \
3   2020-11-27T11:10:09.815282+00:00          0.8499       NaN        NaN   
5   2021-06-03T18:43:52.421011+00:00          0.1247       NaN        NaN   
13  2021-05-08T18:57:27.198023+00:00          0.8575       NaN        NaN   
30  2020-12-10T21:52:05.027569+00:00          0.9928       NaN        NaN   
41  2020-12-18T20:47:06.351547+00:00          0.9558      35.9        5.3   

     age  gender respiratory_condition fever_muscle_pain       status  \
3   73.0    male                 False             False      healthy   
5   14.0    male                 False             False  symptomatic   
13  42.0  female                 False             False      healthy   
30  26.0  female                 False             False  symptomatic   
41   NaN     NaN                   NaN               NaN          NaN   

                                    file_name  \
3   afd8d2b2-b4f1-49e0-ace8-29af9eb8f5e6.json   


In [ ]:
# Keep all normal and cold
df_filtered = df[df["status"].isin(["healthy", "symptomatic"])].copy()
df_filtered["label"] = df_filtered["status"].map({"healthy": "normal", "symptomatic": "cold"})

# Add filepaths (already exists if you did before)
# df_filtered["filepath"] = df_filtered["uuid"].apply(lambda x: os.path.join(audio_path, f"{x}.wav"))

print("Total samples:")
print(df_filtered["label"].value_counts())


Total samples:
label
normal    1533
cold       606
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df_filtered["filepath"]
y = df_filtered["label"]

le = LabelEncoder()
y_enc = le.fit_transform(y)  # normal -> 0, cold -> 1

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_enc, test_size=0.30, stratify=y_enc, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print("Training samples:", len(X_train))
print("Validation samples:", len(X_val))
print("Test samples:", len(X_test))


Training samples: 1497
Validation samples: 321
Test samples: 321


In [ ]:
import librosa
import numpy as np

n_mfcc = 40
max_len = 400

def extract_mfcc_flat(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0,0),(0,pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc.flatten()

# Extract features for all splits
X_train_feat = np.array([extract_mfcc_flat(f) for f in X_train])
X_val_feat   = np.array([extract_mfcc_flat(f) for f in X_val])
X_test_feat  = np.array([extract_mfcc_flat(f) for f in X_test])


Streaming output truncated to the last 5000 lines.
/tmp/ipython-input-736882341.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-736882341.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-736882341.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=1600

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train_feat, y_train)

y_pred = rf.predict(X_test_feat)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)
print(classification_report(y_test, y_pred))


Test Accuracy: 0.719626168224299
              precision    recall  f1-score   support

           0       1.00      0.01      0.02        91
           1       0.72      1.00      0.84       230

    accuracy                           0.72       321
   macro avg       0.86      0.51      0.43       321
weighted avg       0.80      0.72      0.61       321



In [ ]:
import joblib

# Save the model to a file
model_filename = "rf_normal_cold_model.pkl"
joblib.dump(rf, model_filename)

print(f"Model saved as {model_filename}")


Model saved as rf_normal_cold_model.pkl
